# Importando Bibliotecas

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import time

# Conexão com Banco MySQL

In [2]:
# Carregar variáveis do arquivo .env
load_dotenv(dotenv_path="/home/jovyan/app/scripts/.env")

# Recuperar as variáveis do banco de dados
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# Exibir as variáveis (opcional, para verificação)
print(f"Conectando ao banco {database} em {host}:{port} com o usuário {user}")

Conectando ao banco ada_tech em 129.148.25.96:3306 com o usuário alunos_ada


In [3]:
# Criar a engine de conexão
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Querys a Partir de Pandas + MySQL

## Funções e Definições

In [4]:
def get_time(query, engine, nome):
    start_time = time.time()

    df = pd.read_sql(query, con=engine)

    df.to_csv(f"/home/jovyan/app/data/gold/csv/{nome}.csv")

    display(df)

    end_time = time.time()
    
    tempo_query = end_time - start_time

    return tempo_query

def get_time_query3(engine, nome):
    start_time = time.time()

    # Divida os dados em intervalos
    step = 100000  # Tamanho do intervalo
    start = 0      # Valor inicial
    end = start + step

    # Lista para resultados
    df_resultados = pd.DataFrame(columns=["codigo_ibge", "latitude", "longitude", "in_area"])

    while True:
        print(f"Query das linhas {start} até {end}")
        
        # Query SQL
        query = f"""
        SELECT 
            codigo_ibge, 
            latitude, 
            longitude
            
        FROM ada_tech.prof_car_local
        WHERE tipo_imovel_rural = 'IRU'
        LIMIT {step} OFFSET {start};"""

        # Carregar os dados em um DataFrame
        
        # Leia o chunk
        chunk = pd.read_sql_query(query, con=engine)

        # Quebra do loop se nenhum dado for retornado
        if chunk.empty:
            break
        else:
            chunk["in_area"] = chunk.apply(lambda row: is_point_in_path(row["longitude"], row["latitude"], poly), axis=1)

            df_resultados = pd.concat([df_resultados, chunk[chunk["in_area"] == True]], ignore_index=True)

        start += step
        
        end += step

    df_resultados.to_csv(f"/home/jovyan/app/data/gold/csv/{nome}.csv")

    display(df_resultados)

    end_time = time.time()
    
    tempo_query = end_time - start_time

    return tempo_query


def is_point_in_path(x: float, y: float, poly: list[tuple[float, float]]) -> bool:
    """Determine if the point is on the path, corner, or boundary of the polygon

    Args:
    x -- The x coordinates of point.
    y -- The y coordinates of point.
    poly -- a list of tuples [(x, y), (x, y), ...]

    Returns:
        True if the point is in the path or is a corner or on the boundary"""
    
    c = False

    for i in range(len(poly)):
        ax, ay = poly[i]
        bx, by = poly[i - 1]
        if (x == ax) and (y == ay):
            # point is a corner
            return True
        if (ay > y) != (by > y):
            slope = (x - ax) * (by - ay) - (bx - ax) * (y - ay)
            if slope == 0:
                # point is on boundary
                return True
            if (slope < 0) != (by < ay):
                c = not c

    return c

# Poligono
poly = [(-53.5325072, -19.4632582), (-51.0495971, -19.1625841), (-51.3734501, -16.1924262), (-53.8181518, -16.4010783), (-53.5325072, -19.4632582)]

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [5]:
query = """
SELECT 
    SUM(area_do_imovel) AS soma_area_imovel 
    
    FROM ada_tech.prof_car_local 
    
    WHERE uf IN ('MS', 'MT')
    
    GROUP BY uf
    
    ORDER BY soma_area_imovel desc;
"""

tempo_1 = get_time(query, engine, "query_1")
tempo_1

,soma_area_imovel
0,8.458709e+07
1,3.632188e+07


4.200398683547974

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [6]:
# Query SQL
query = """
SELECT DISTINCT
        uf,
        codigo_ibge

FROM ada_tech.prof_car_local 

WHERE uf IN ('SP','MG','ES','RJ');"""

tempo_2 = get_time(query, engine, "query_2")
tempo_2

,uf,codigo_ibge
0,RJ,3302205
1,RJ,3304300
2,MG,3156700
3,MG,3147907
4,MG,3117108
...,...,...
1659,SP,3513801
1660,SP,3505708
1661,SP,3552809
1662,SP,3534401


9.475553274154663

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [7]:
tempo_3 = get_time_query3(engine, "query_3")
tempo_3

Query das linhas 0 até 100000
Query das linhas 100000 até 200000
Query das linhas 200000 até 300000
Query das linhas 300000 até 400000
Query das linhas 400000 até 500000
Query das linhas 500000 até 600000
Query das linhas 600000 até 700000
Query das linhas 700000 até 800000
Query das linhas 800000 até 900000
Query das linhas 900000 até 1000000
Query das linhas 1000000 até 1100000
Query das linhas 1100000 até 1200000
Query das linhas 1200000 até 1300000
Query das linhas 1300000 até 1400000
Query das linhas 1400000 até 1500000
Query das linhas 1500000 até 1600000
Query das linhas 1600000 até 1700000
Query das linhas 1700000 até 1800000
Query das linhas 1800000 até 1900000
Query das linhas 1900000 até 2000000
Query das linhas 2000000 até 2100000
Query das linhas 2100000 até 2200000
Query das linhas 2200000 até 2300000
Query das linhas 2300000 até 2400000
Query das linhas 2400000 até 2500000
Query das linhas 2500000 até 2600000
Query das linhas 2600000 até 2700000
Query das linhas 2700000 

,codigo_ibge,latitude,longitude,in_area
0,5204409,-17.118992,-51.595982,True
1,5204409,-17.123236,-51.611193,True
2,5204409,-17.227269,-52.061138,True
3,5204409,-17.092364,-51.598947,True
4,5204409,-17.117028,-51.459311,True
...,...,...,...,...
23911,5213103,-17.385358,-52.141239,True
23912,5201454,-18.227415,-51.422426,True
23913,5201454,-18.239369,-51.432668,True
23914,5002902,-19.108579,-51.726561,True


319.9095666408539

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [8]:
# Query SQL
query = """
SELECT
    YEAR(data_inscricao) AS ano_inscricao, 
    COUNT(*) AS propriedade_cadastradas

FROM ada_tech.prof_car_local 

GROUP BY ano_inscricao

ORDER BY ano_inscricao;
"""

tempo_4 = get_time(query, engine, "query_4")
tempo_4

,ano_inscricao,propriedade_cadastradas
0,NaN,12
1,2013.0,743
2,2014.0,227468
3,2015.0,1240178
4,2016.0,1828786
5,2017.0,794779
6,2018.0,724856
7,2019.0,746570
8,2020.0,463322
9,2021.0,443942


21.658092975616455

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [9]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   (pcl.area_remanescente_vegetacao_nativa / pcl.area_do_imovel * 100) as percentual_area_nativa

FROM ada_tech.prof_car_local pcl;
""" 

tempo_5 = get_time(query, engine, "query_5")
tempo_5

,codigo_ibge,area_do_imovel,area_remanescente_vegetacao_nativa,percentual_area_nativa
0,5214408,119.6326,6.432028,5.376484
1,4210803,7.5340,5.517854,73.239370
2,5214903,19.4883,19.488263,99.999812
3,5219308,22.9340,1.502150,6.549881
4,4106407,10.9560,0.000000,0.000000
...,...,...,...,...
6839099,4125704,121.0585,NaN,NaN
6839100,3160900,2.2491,NaN,NaN
6839101,5106182,25.1107,NaN,NaN
6839102,2615805,51.2083,NaN,NaN


67.99589943885803

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

* AST – Assentamentos
* CAR – Cadastro Ambiental Rural
* CCU – Contrato de Concessão de Uso
* CDRU – Concessão de Direito Real de Uso
* DD – Diretoria de Desenvolvimento
* IRU – Imóveis Rurais
* MLC – Módulo Lote CAR
* PA – Projeto de Assentamento
* PCT – Povos e Comunidades Tradicionais
* PRA – Programa de Regularização Ambiental
* PNRA – Programa Nacional de Reforma Agrária
* SICAR – Sistema de Cadastro Ambiental Rural
* SR – Superintendência Regional
* TCMS – Termo de Compromisso de Manutenção de Sigilo
* TD – Título Definitivo
* TED – Termo de Execução Descentralizada
* URL – Uniform Resource Locator

In [10]:
query = """
SELECT DISTINCT
    pcl.uf,
    COUNT(pcl.tipo_imovel_rural) AS qnt_propriedades_rurais

FROM ada_tech.prof_car_local pcl 

WHERE pcl.tipo_imovel_rural = 'IRU'

GROUP BY pcl.uf;
"""

tempo_6 = get_time(query, engine, "query_6")
tempo_6

,uf,qnt_propriedades_rurais
0,GO,198524
1,SC,374768
2,PR,495650
3,RJ,58848
4,MA,272010
5,RS,606014
6,PI,254577
7,PE,338392
8,DF,17335
9,MG,988923


11.581441402435303

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [11]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    (SQRT(
        POWER(latitude - (-15.796943053171708), 2) + 
        POWER(longitude - (-47.891638482569476), 2) 
        ) * 111) AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM ada_tech.prof_car_local pcl

    ORDER BY area_do_imovel DESC

    LIMIT 1
) AS subquery
"""

tempo_7 = get_time(query, engine, "query_7")
tempo_7

,max_area,latitude,longitude,distancia_brasilia
0,2.420079e+06,-5.475619,-68.888188,2596.984499


5.193731307983398

## Consulta 8

In [12]:
# /* Consulta 8
# 	Faça a média de área entre todas as propriedades. Calcule quantas 
# 	propriedades por estado, estão acima da média. */

# Query SQL
query = """
SELECT DISTINCT
    uf,
    COUNT(pcl.registro_car) AS qnt_propriedades,
    AVG(pcl.area_do_imovel) AS media_area

FROM ada_tech.prof_car_local pcl

GROUP BY uf

HAVING media_area > (SELECT AVG(area_do_imovel) FROM ada_tech.prof_car_local);"""

tempo_8 = get_time(query, engine, "query_8")
tempo_8

,uf,qnt_propriedades,media_area
0,GO,199690,164.781607
1,MA,273642,112.599726
2,AM,73642,968.969346
3,RR,22687,390.411597
4,AP,10116,511.920164
5,AC,45141,306.883235
6,TO,85537,293.615894
7,MT,170205,496.971824
8,RO,147287,108.260902
9,PA,279598,293.775381


15.687904834747314

In [13]:
total_pandas_mysql = tempo_1 + tempo_2 + tempo_3 + tempo_4 + tempo_5 + tempo_6 + tempo_7 + tempo_8
total_pandas_mysql

455.702588558197